# Importing Libraries and Creating Session

In [28]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import shutil
import os
import pyspark.sql.functions as F
import shutil
from os import path

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \

spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

# Cleaning the Data and Creating the Dictionary 

In [29]:
def removeChar(text):
    
    remove = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
    lower_result = text.lower()

    for char in remove:
        lower_result = lower_result.replace(char, '')

    return lower_result

def splitRDD(rdd):
    return rdd.map(removeChar) \
            .flatMap(lambda text: text.split(' ')) \
            .filter(lambda text: text != '')

path = 'dataset/*'

rdd = spark.sparkContext.textFile(path)
final_rdd = splitRDD(rdd)
dictionary = final_rdd.distinct() \
            .zipWithIndex()


df = spark.createDataFrame(dictionary, ["word", "wordId"])


# Applying Inverted Index

In [34]:
path = 'dataset/'
files = []

for name in os.listdir(path):
    file = os.path.join(path, name)

    if os.path.isfile(file):
        files.append(int(name))



list_files = []

for file in files:
    rdd = spark.sparkContext.textFile(path + str(file))
    rdd = splitRDD(rdd).distinct()
    rdd = rdd.map(lambda word: (word, file))
    list_files.append(rdd)


pre_agg_df = files_df.join(
    dictionary_df, dictionary_df.word == files_df.word, how='left')



files_and_rdd = spark.sparkContext.union(list_of_rdd)
files_and_df = spark.createDataFrame(files_and_rdd, ['word', 'docId'])

final_df = joinFilesDict(files_and_df, df)

inverted_index = final_df \
    .groupBy(F.col('wordId')) \
    .agg(F.sort_array(F.collect_list(F.col('docId'))).alias('docId')) \
    .orderBy(F.col('wordId'))


print(inverted_index_df.head(5))




[Row(wordId=0, docId=[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]), Row(wordId=1, docId=[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]), Row(wordId=2, docId=[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]), Row(wordId=3, docId=[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]), Row(wordId=4, docId=[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25])]
